## Assumptions

In the first notebook we have established the structure and contents of the `modelnet40v1` dataset.

Now, we will attempt to create a model that is capable of determining the type of the object based on all images of an object instance. The type of architecture that allows just that is called a *Multi-View Convolutional Network (`MVCNN`)* - you can read about it in more detail [in this arXiv paper](https://arxiv.org/pdf/1505.00880.pdf). In short: it allows feature extraction from multiple images, pools the individual image feature vectors using a symmetrical function and finally makes a prediction on the pooled vector. Because we are using a symmetrical function for pooling, the number of images per instance is arbitrary and can change from instance to instance.

![mvcnn_figure](./data/mvcnn_figure.JPG)

In our case, this experiment will be performed with the following configuration - we'll use:
* `ResNet50` pretrained on the `ImageNet` dataset as the backbone feature extractor for the individual images
* `mean` as the symmetrical pooling function for the individual vectors 
* `dropout` in the layers responsible for pooling and prediction
* `Adam` as the optimizer
* `F1-Macro` calculated on the validation set as the driving metric to reveal our champion model

In addition, because the experiment will be performed on a local machine with no access to a reasonable GPU, we will make some simplifications in the training pipeline:
* the number of classes in training will be limited to just 4: `airplane`, `bathtub`, `bed` and `bench`
* the pretrained feature extractor weights will remain frozen throughout the training
* at training time we'll use the batch dimension of the input to provide the instance images - instances will be forwarded through the net one by one, i.e. the *de facto* batch size will be 1

## Imports

In [1]:
%load_ext autoreload
%autoreload 2

from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer

from dataset_classes.mvcnn_data_module import MVCNNDataModule
from model_classes.mvcnn import MVCNNClassifier
from model_classes.callbacks import UnfreezePretrainedWeights

/mnt/d/coding/git/TomaszKaleczyc/MVCNN_training/environment/mvcnn_train/lib/python3.6/site-packages/pytorch_lightning/metrics/__init__.py:44: LightningDeprecationWarning: `pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package (https://github.com/PyTorchLightning/metrics) since v1.3 and will be removed in v1.5
  "`pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package"


## Parameters

In [2]:
NUM_CLASSES = 4
LEARNING_RATE = 1e-3
LEARNING_RATE_REDUCTION_FACTOR = 1e3
NUM_EPOCHS = 20
NUM_EPOCHS_FREEZE_PRETRAINED = 20
BATCH_SIZE = 1
DROPOUT_RATE = 0.3
SAVE_PATH = '../output'

## Class initialization

In [3]:
data_module = MVCNNDataModule(NUM_CLASSES, BATCH_SIZE)

Dataset type: TRAIN
------------------------------------------------------------
Class name: AIRPLANE
Total number of instances: 80
Total number of images: 960
------------------------------------------------------------
Class name: BATHTUB
Total number of instances: 80
Total number of images: 960
------------------------------------------------------------
Class name: BED
Total number of instances: 80
Total number of images: 960
------------------------------------------------------------
Class name: BENCH
Total number of instances: 80
Total number of images: 960
Dataset type: TEST
------------------------------------------------------------
Class name: AIRPLANE
Total number of instances: 20
Total number of images: 240
------------------------------------------------------------
Class name: BATHTUB
Total number of instances: 20
Total number of images: 240
------------------------------------------------------------
Class name: BED
Total number of instances: 20
Total number of images: 

In [4]:
model = MVCNNClassifier(
    num_classes=NUM_CLASSES,
    learning_rate=LEARNING_RATE,
    num_epochs_freeze_pretrained=NUM_EPOCHS_FREEZE_PRETRAINED,
    dropout_rate=DROPOUT_RATE,
    )

Feature extractor weights frozen


In [5]:
callbacks = [
    ModelCheckpoint(
        filename='mvcnn-{epoch}-{validation/f1:.3f}',
        monitor='validation/f1', 
        save_top_k=3,
        verbose=True, 
        mode='max'
        ),
    UnfreezePretrainedWeights(LEARNING_RATE_REDUCTION_FACTOR),
]

In [6]:
trainer = Trainer(
    max_epochs=NUM_EPOCHS,
    fast_dev_run=False,
    default_root_dir=SAVE_PATH,
    callbacks=callbacks
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


## Model training

In [7]:
trainer.fit(
    model,
    train_dataloader=data_module.train_dataloader(),
    val_dataloaders=data_module.val_dataloader()
)


  | Name                  | Type             | Params
-----------------------------------------------------------
0 | feature_extractor     | FeatureExtractor | 25.6 M
1 | image_vector_creator  | Sequential       | 512 K 
2 | predictor             | Sequential       | 74.5 K
3 | avg_metric            | AverageMeter     | 0     
4 | eval_metric           | F1               | 0     
5 | secondary_eval_metric | Accuracy         | 0     
-----------------------------------------------------------
586 K     Trainable params
25.6 M    Non-trainable params
26.1 M    Total params
104.576   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Eval stats for VALIDATION
TP per class: [0 0 0 0] Average: 0.00
TN per class: [0 0 2 2] Average: 1.00
FP per class: [0 2 0 0] Average: 0.50
FN per class: [2 0 0 0] Average: 0.50
ACCURACY per class: [0. 0. 1. 1.] Average: 0.50
PRECISION per class: [0. 0. 0. 0.] Average: 0.00
RECALL per class: [0. 0. 0. 0.] Average: 0.00
F1 per class: [0. 0. 0. 0.] Average: 0.00
Avg Loss na validation tensor(1.4405)
F1 - validation tensor(0.)
Accuracy - validation tensor(0.7500)


Training: 0it [00:00, ?it/s]

Eval stats for TRAIN
TP per class: [48 26 23 18] Average: 28.75
TN per class: [200 168 194 193] Average: 188.75
FP per class: [40 72 46 47] Average: 51.25
FN per class: [32 54 57 62] Average: 51.25
ACCURACY per class: [0.775    0.60625  0.678125 0.659375] Average: 0.68
PRECISION per class: [0.54545455 0.26530612 0.33333333 0.27692308] Average: 0.36
RECALL per class: [0.6    0.325  0.2875 0.225 ] Average: 0.36
F1 per class: [0.57142857 0.29213483 0.30872483 0.24827586] Average: 0.36
Avg Loss na train tensor(1.2140, grad_fn=<DivBackward0>)
F1 - train tensor(0.3551)
Accuracy - train tensor(0.7672)


...

Epoch 18, global step 6079: validation/f1 was not in top 3


Eval stats for VALIDATION
TP per class: [18  8 11 17] Average: 13.50
TN per class: [58 60 54 42] Average: 53.50
FP per class: [ 2  0  6 18] Average: 6.50
FN per class: [ 2 12  9  3] Average: 6.50
ACCURACY per class: [0.95   0.85   0.8125 0.7375] Average: 0.84
PRECISION per class: [0.9        1.         0.64705882 0.48571429] Average: 0.76
RECALL per class: [0.9  0.4  0.55 0.85] Average: 0.68
F1 per class: [0.9        0.57142857 0.59459459 0.61818182] Average: 0.67
Avg Loss na validation tensor(4.2849)
F1 - validation tensor(0.6711)
Accuracy - validation tensor(0.8375)
Eval stats for TRAIN
TP per class: [80 79 77 77] Average: 78.25
TN per class: [240 239 238 236] Average: 238.25
FP per class: [0 1 2 4] Average: 1.75
FN per class: [0 1 3 3] Average: 1.75
ACCURACY per class: [1.       0.99375  0.984375 0.978125] Average: 0.99
PRECISION per class: [1.         0.9875     0.97468354 0.95061728] Average: 0.98
RECALL per class: [1.     0.9875 0.9625 0.9625] Average: 0.98
F1 per class: [1.     

Validating: 0it [00:00, ?it/s]

Epoch 19, global step 6399: validation/f1 was not in top 3


Eval stats for VALIDATION
TP per class: [17 13  9 17] Average: 14.00
TN per class: [58 60 53 45] Average: 54.00
FP per class: [ 2  0  7 15] Average: 6.00
FN per class: [ 3  7 11  3] Average: 6.00
ACCURACY per class: [0.9375 0.9125 0.775  0.775 ] Average: 0.85
PRECISION per class: [0.89473684 1.         0.5625     0.53125   ] Average: 0.75
RECALL per class: [0.85 0.65 0.45 0.85] Average: 0.70
F1 per class: [0.87179487 0.78787879 0.5        0.65384615] Average: 0.70
Avg Loss na validation tensor(2.8940)
F1 - validation tensor(0.7034)
Accuracy - validation tensor(0.8500)
